In [1]:
# Crypto heatmap

In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2

In [3]:
url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=10&e=Coinbase"
f = requests.get(url)
ipdata = f.json()
pd.DataFrame(ipdata['Data']).head(5)

,time,close,high,low,open,volumefrom,volumeto
0,1583366400,9070.17,9174.24,8759.99,8760.66,9805.85,8.850863e+07
1,1583452800,9158.51,9182.08,9000.01,9070.17,6422.13,5.839648e+07
2,1583539200,8901.37,9214.67,8850.47,9158.51,5965.47,5.378014e+07
3,1583625600,8037.76,8901.37,8002.20,8901.37,15340.83,1.290085e+08
4,1583712000,7934.52,8187.03,7630.00,8037.76,24964.13,1.959799e+08


In [4]:
def get_data(date):
    """query API for 2000 days historical price data starting from "date"."""
    url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=2000&toTs={}".format(
        date)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

In [5]:
def get_df(from_date, to_date):
    date = to_date
    holder = []
    # while earliest date returned is later than the earliest data requested, keep querying API # and adding results to list
    while date > from_date:
        data = get_data(date)
        holder.append(pd.DataFrame(data['Data']))
        date = data['TimeFrom']
        # join together all api queries from list
        df = pd.concat(holder, axis=0)
        # remove data points from before from_date
        df = df[df['time'] > from_date]
        # convert to timestamp
        df['time'] = pd.to_datetime(df['time'], unit='s')
        # make datafrom index the time
        df.set_index('time', inplace=True)
        # sort in time order
        df.sort_index(ascending=False, inplace=True)
        return df

In [6]:
df = get_df(1583038800, 1584307800)

In [7]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(df[['low', 'close', 'high']])
ax.set_ylabel('Price$')
ax.set_xlabel('Date')
ax.legend(['Low', 'Close', 'High'])
plt.show()

In [8]:
def get_data_spec(coin, date, time_period):
    """query api for historical data starting from "date"."""
    url = "https://min-api.cryptocompare.com/data/{}?fsym={}&tsym=USD&Limit=2000&toTs={}".format(
        time_period, coin, date)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

In [9]:
def get_df_spec(time_period, coin, from_date, to_date):
    """ get historical price data between dates. If further apart query multiple times."""
    date = to_date
    holder = []
    while date > from_date:
        data = get_data_spec(coin, date, time_period)
        holder.append(pd.DataFrame(data['Data']))
        date = data['TimeFrom']
    df = pd.concat(holder, axis=0)
    df = df[df['time'] > from_date]
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    df.sort_index(ascending=False, inplace=True)
    df.rename(columns={'close': coin}, inplace=True)
    return df[coin]

In [10]:
coins = ['BTC', 'ETH', 'LTC', 'DOGE', 'XRP', 'EOS', 'ADA']

In [11]:
holder = []
from_date = 1583038800 #03-01-2020
to_date = 1584307800 #03-15-2020
time_period = 'histoday'

for coin in coins:
    holder.append(get_df_spec(time_period, coin, from_date, to_date))
df = pd.concat(holder, axis=1)

In [12]:
# convert each column of df to be rate of return instead of price
df = df.divide(df.shift()) - 1
df_norm_col = (df - df.mean()) / df.std()

In [13]:
fig = plt.figure(figsize=(15, 10))
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True, linewidth=0.5, cmap=cmap)
plt.show()